# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#507 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846821475033                   -0.70    4.0
  2   -7.852323211886       -2.26       -1.53    1.0
  3   -7.852645966447       -3.49       -2.53    3.0
  4   -7.852646676423       -6.15       -3.34    2.5
  5   -7.852646685842       -8.03       -4.74    1.0
  6   -7.852646686727       -9.05       -5.25    3.8
  7   -7.852646686729      -11.59       -5.87    1.0
  8   -7.852646686730      -12.37       -7.32    2.0
  9   -7.852646686730      -14.45       -7.55    3.0
 10   -7.852646686730   +  -15.05       -8.17    1.5
 11   -7.852646686730   +  -14.27       -9.68    2.5
 12   -7.852646686730      -14.45      -10.07    2.2
 13   -7.852646686730      -14.75      -11.04    2.5
 14   -7.852646686730   +  -14.75      -11.71    2.2
 15   -7.852646686730      -14.75      -12.54    2.8


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846836113874                   -0.70           4.2
  2   -7.852524770350       -2.24       -1.62   0.80    4.8
  3   -7.852610414457       -4.07       -2.78   0.80    5.2
  4   -7.852646570879       -4.44       -3.57   0.80    7.5
  5   -7.852646683762       -6.95       -4.72   0.80    8.8
  6   -7.852646686706       -8.53       -5.19   0.80   11.2
  7   -7.852646686729      -10.65       -6.09   0.80   12.2
  8   -7.852646686730      -12.05       -7.08   0.80   13.0
  9   -7.852646686730      -13.35       -8.18   0.80   15.0
 10   -7.852646686730   +  -15.05       -9.49   0.80   17.0
 11   -7.852646686730      -14.75      -10.17   0.80   19.5
 12   -7.852646686730   +  -14.75      -10.73   0.80   20.2
 13   -7.852646686730   +  -14.57      -11.93   0.80   21.2
 14   -7.852646686730      -14.35      -12.62   0.80   24.5


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.458703e+01     3.068649e+00
 * time: 0.48297619819641113
     1     1.443960e+00     1.730799e+00
 * time: 0.7332320213317871
     2    -1.908458e+00     2.071020e+00
 * time: 0.7627291679382324
     3    -3.659795e+00     1.843891e+00
 * time: 0.8050150871276855
     4    -5.258848e+00     1.425561e+00
 * time: 0.8474941253662109
     5    -6.733860e+00     8.567921e-01
 * time: 0.8898751735687256
     6    -7.331126e+00     4.005871e-01
 * time: 0.9321930408477783
     7    -7.532798e+00     2.040724e-01
 * time: 0.9614620208740234
     8    -7.644429e+00     1.499170e-01
 * time: 0.9909131526947021
     9    -7.740599e+00     1.678728e-01
 * time: 1.0206630229949951
    10    -7.796063e+00     1.039214e-01
 * time: 1.050159215927124
    11    -7.833787e+00     5.107705e-02
 * time: 1.0794901847839355
    12    -7.845320e+00     4.149330e-02
 * time: 1.108699083328247
    13    -7.850987e+00     3.010337e-02
 * time: 1.13787603378

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846823629066                   -0.70    4.2
  2   -7.852324482527       -2.26       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686707                   -2.54
  2   -7.852646686730      -10.65       -5.88
  3   -7.852646686730   +  -15.05      -12.54
|ρ_newton - ρ_scf|  = 2.2041796367313623e-13
|ρ_newton - ρ_scfv| = 4.186317894886605e-14
|ρ_newton - ρ_dm|   = 4.3679194157154396e-10
